In [2]:
from sklearn.datasets import load_wine

In [4]:
wine_data = load_wine()
type(wine_data)

sklearn.utils.Bunch

In [6]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [7]:
print(wine_data['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [8]:
feat_data = wine_data['data']
target = wine_data['target']

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(feat_data, target,test_size= 0.3,
                                                    random_state= 101 )

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scale_x_train= scaler.fit_transform(x_train)
scalex_test = scaler.transform(x_test)

In [14]:
from tensorflow import estimator
import tensorflow as tf

In [15]:
x_train.shape

(124, 13)

In [16]:
feat_fn = [tf.feature_column.numeric_column('x', shape=[13])]

In [17]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13],
                                    feature_columns= feat_fn,
                                    n_classes=3,
                                    optimizer=tf.train.GradientDescentOptimizer(learning_rate= 0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Gaurav\\AppData\\Local\\Temp\\tmp4j5nnwe3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C68585ABE0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scale_x_train}, y= y_train,
                                          shuffle=True,
                                          batch_size=10,
                                          num_epochs=5)

In [31]:
deep_model.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Gaurav\AppData\Local\Temp\tmp4j5nnwe3\model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 62 into C:\Users\Gaurav\AppData\Local\Temp\tmp4j5nnwe3\model.ckpt.
INFO:tensorflow:loss = 8.81156, step = 63
INFO:tensorflow:Saving checkpoints for 124 into C:\Users\Gaurav\AppData\Local\Temp\tmp4j5nnwe3\model.ckpt.
INFO:tensorflow:Loss for final step: 2.964017.


In [32]:
input_eval_fn = estimator.inputs.numpy_input_fn(x= {'x':scalex_test}, shuffle= False)

In [33]:
pred = list(deep_model.predict(input_fn=input_eval_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Gaurav\AppData\Local\Temp\tmp4j5nnwe3\model.ckpt-124
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [38]:
pred

[{'logits': array([ 0.60416317, -0.6553995 , -0.44835272], dtype=float32),
  'probabilities': array([0.61243117, 0.17379454, 0.21377428], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([ 0.60732925, -0.6389125 , -0.4567824 ], dtype=float32),
  'probabilities': array([0.61251324, 0.17614874, 0.21133803], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-1.1497335, -2.8057544,  2.1975386], dtype=float32),
  'probabilities': array([0.03376555, 0.00644574, 0.9597887 ], dtype=float32),
  'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object)},
 {'logits': array([ 0.5554501 , -0.90802383, -0.3193069 ], dtype=float32),
  'probabilities': array([0.606651  , 0.14039779, 0.25295123], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-1.250076 , -3.1113474,  2.4090278],

In [34]:
predictions = [ i['class_ids'][0] for i in pred]

In [35]:
predictions

[0,
 0,
 2,
 0,
 2,
 1,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 1,
 0]

In [36]:
from sklearn.metrics import confusion_matrix, classification_report

In [37]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.83      1.00      0.90        19
           1       1.00      0.77      0.87        22
           2       0.93      1.00      0.96        13

   micro avg       0.91      0.91      0.91        54
   macro avg       0.92      0.92      0.91        54
weighted avg       0.92      0.91      0.91        54

